In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from models_def import cnn2d,cnn_v1,cnn_v2,cudnn_gru,lstm_v1,cnn_gru,gru_v1
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import text, sequence

model_defines = [
#     [cnn2d,'cnn2d'],
#     [cnn_v1,'cnn_v1'],
#     [cnn_v2,'cnn_v2'],
#    [cudnn_gru,'cudnn_gru'],
    [lstm_v1,'lstm_v1'],
    [cnn_gru,'cnn_gru'],
    [gru_v1,'gru_v1']
]

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [2]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
word_vec_dict = {}
with open('../wiki.multi.fr.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v

with open('../wiki.multi.de.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v

with open('../wiki.multi.en.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

(159571, 150) (153164, 150)
stonewall 44142
edmonton 15575
kalogerakou 126872
mcp 146121
p754 169036
hpvpp 104509
435713
Preparing embedding matrix
Null word embeddings: 4146


In [3]:
from sklearn.metrics import log_loss,accuracy_score
from sklearn.model_selection import KFold
from keras import backend as K
import pickle
from keras.models import load_model
import gc

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def kf_train(model_func,fold_cnt=3,rnd=1,epo=10,batch=64):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = model_func(maxlen,nb_words,EMBEDDING_DIM,embedding_matrix,trainable_flag=False,comp=True)
        batch_size = batch
        epochs = epo
        file_path="best_model.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), callbacks=callbacks_list)
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
        
        # clear
        del model
        gc.collect()
        K.clear_session()
        
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    eval_val(y,train_pred)
    return train_pred, test_pred

rnd_init = 1
for model_func,model_name in model_defines:
    for fold_k in [3,4]:
        print('==================================')
        if 'cnn' in model_name:
            epo = 10
            batch = 64
        else:
            epo = 8
            batch = 256
        train_pred,test_pred = kf_train(model_func,fold_k,rnd_init,epo,batch)
        output_feat = '../features/muse_{}_{}_feat.pkl'.format(model_name,fold_k)
        with open(output_feat,'wb') as fout:
            pickle.dump([train_pred,test_pred],fout)
        print(output_feat,'done')
        rnd_init += 1
        




Train on 106380 samples, validate on 53191 samples
Epoch 1/8
106380/106380 [==============================] - 172s 2ms/step - loss: 0.1168 - acc: 0.9664 - val_loss: 0.0620 - val_acc: 0.9778
Epoch 2/8
106380/106380 [==============================] - 170s 2ms/step - loss: 0.0605 - acc: 0.9789 - val_loss: 0.0547 - val_acc: 0.9802
Epoch 3/8
106380/106380 [==============================] - 170s 2ms/step - loss: 0.0566 - acc: 0.9799 - val_loss: 0.0518 - val_acc: 0.9807
Epoch 4/8
106380/106380 [==============================] - 170s 2ms/step - loss: 0.0543 - acc: 0.9804 - val_loss: 0.0507 - val_acc: 0.9811
Epoch 5/8
106380/106380 [==============================] - 170s 2ms/step - loss: 0.0528 - acc: 0.9811 - val_loss: 0.0496 - val_acc: 0.9814
Epoch 6/8
106380/106380 [==============================] - 170s 2ms/step - loss: 0.0512 - acc: 0.9814 - val_loss: 0.0490 - val_acc: 0.9815
Epoch 7/8
106380/106380 [==============================] - 170s 2ms/step - loss: 0.0495 - acc: 0.9816 - val_loss: 0

Epoch 2/8
119678/119678 [==============================] - 178s 1ms/step - loss: 0.0579 - acc: 0.9796 - val_loss: 0.0542 - val_acc: 0.9803
Epoch 3/8
119678/119678 [==============================] - 178s 1ms/step - loss: 0.0547 - acc: 0.9804 - val_loss: 0.0533 - val_acc: 0.9806
Epoch 4/8
119678/119678 [==============================] - 178s 1ms/step - loss: 0.0526 - acc: 0.9810 - val_loss: 0.0500 - val_acc: 0.9812
Epoch 5/8
119678/119678 [==============================] - 178s 1ms/step - loss: 0.0513 - acc: 0.9813 - val_loss: 0.0485 - val_acc: 0.9817
Epoch 6/8
119678/119678 [==============================] - 178s 1ms/step - loss: 0.0495 - acc: 0.9817 - val_loss: 0.0470 - val_acc: 0.9819
Epoch 7/8
119678/119678 [==============================] - 178s 1ms/step - loss: 0.0479 - acc: 0.9822 - val_loss: 0.0459 - val_acc: 0.9822
Epoch 8/8
119678/119678 [==============================] - 178s 1ms/step - loss: 0.0467 - acc: 0.9825 - val_loss: 0.0449 - val_acc: 0.9824
Train on 119678 samples, va

119679/119679 [==============================] - 178s 1ms/step - loss: 0.0511 - acc: 0.9813 - val_loss: 0.0502 - val_acc: 0.9818
Epoch 5/8
119679/119679 [==============================] - 178s 1ms/step - loss: 0.0494 - acc: 0.9819 - val_loss: 0.0470 - val_acc: 0.9824
Epoch 6/8
119679/119679 [==============================] - 178s 1ms/step - loss: 0.0481 - acc: 0.9820 - val_loss: 0.0463 - val_acc: 0.9826
Epoch 7/8
119679/119679 [==============================] - 178s 1ms/step - loss: 0.0467 - acc: 0.9824 - val_loss: 0.0458 - val_acc: 0.9827
Epoch 8/8
119679/119679 [==============================] - 178s 1ms/step - loss: 0.0458 - acc: 0.9827 - val_loss: 0.0446 - val_acc: 0.9832
-------------------------------
0 0.0971482752696 0.96358360855
1 0.0227032613585 0.990693797745
2 0.0505491105307 0.981450263519
3 0.0101202565546 0.997067136259
4 0.0639435742995 0.973804764024
5 0.0213950948614 0.992837044325
final 0.0443099288124 0.983239435737
../features/muse_lstm_v1_4_feat.pkl done
Train on

Epoch 3/10
106381/106381 [==============================] - 124s 1ms/step - loss: 0.0449 - acc: 0.9829 - val_loss: 0.0467 - val_acc: 0.9825
Epoch 4/10
106381/106381 [==============================] - 124s 1ms/step - loss: 0.0415 - acc: 0.9841 - val_loss: 0.0437 - val_acc: 0.9835
Epoch 5/10
106381/106381 [==============================] - 124s 1ms/step - loss: 0.0385 - acc: 0.9849 - val_loss: 0.0438 - val_acc: 0.9833
Epoch 6/10
106381/106381 [==============================] - 123s 1ms/step - loss: 0.0361 - acc: 0.9858 - val_loss: 0.0444 - val_acc: 0.9836
Epoch 7/10
106381/106381 [==============================] - 123s 1ms/step - loss: 0.0335 - acc: 0.9868 - val_loss: 0.0464 - val_acc: 0.9835
Epoch 8/10
106381/106381 [==============================] - 123s 1ms/step - loss: 0.0313 - acc: 0.9875 - val_loss: 0.0465 - val_acc: 0.9824
Epoch 9/10
106381/106381 [==============================] - 123s 1ms/step - loss: 0.0295 - acc: 0.9881 - val_loss: 0.0468 - val_acc: 0.9828
Epoch 10/10
106381/1

Train on 119678 samples, validate on 39893 samples
Epoch 1/10
119678/119678 [==============================] - 134s 1ms/step - loss: 0.0700 - acc: 0.9763 - val_loss: 0.0468 - val_acc: 0.9827
Epoch 2/10
119678/119678 [==============================] - 134s 1ms/step - loss: 0.0484 - acc: 0.9819 - val_loss: 0.0439 - val_acc: 0.9834
Epoch 3/10
119678/119678 [==============================] - 134s 1ms/step - loss: 0.0442 - acc: 0.9831 - val_loss: 0.0421 - val_acc: 0.9839
Epoch 4/10
119678/119678 [==============================] - 134s 1ms/step - loss: 0.0411 - acc: 0.9841 - val_loss: 0.0417 - val_acc: 0.9839
Epoch 5/10
119678/119678 [==============================] - 134s 1ms/step - loss: 0.0385 - acc: 0.9850 - val_loss: 0.0418 - val_acc: 0.9840
Epoch 6/10
119678/119678 [==============================] - 134s 1ms/step - loss: 0.0360 - acc: 0.9858 - val_loss: 0.0440 - val_acc: 0.9837
Epoch 7/10
119678/119678 [==============================] - 134s 1ms/step - loss: 0.0336 - acc: 0.9868 - val_

Epoch 8/8
106380/106380 [==============================] - 132s 1ms/step - loss: 0.0453 - acc: 0.9830 - val_loss: 0.0442 - val_acc: 0.9832
Train on 106381 samples, validate on 53190 samples
Epoch 1/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.1024 - acc: 0.9700 - val_loss: 0.0531 - val_acc: 0.9811
Epoch 2/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0559 - acc: 0.9798 - val_loss: 0.0498 - val_acc: 0.9820
Epoch 3/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0521 - acc: 0.9808 - val_loss: 0.0468 - val_acc: 0.9827
Epoch 4/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0497 - acc: 0.9814 - val_loss: 0.0457 - val_acc: 0.9832
Epoch 5/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0483 - acc: 0.9819 - val_loss: 0.0445 - val_acc: 0.9835
Epoch 6/8
106381/106381 [==============================] - 133s 1ms/step - loss: 0.0468 - acc: 0.9823 - val_loss: 0

119678/119678 [==============================] - 140s 1ms/step - loss: 0.0965 - acc: 0.9708 - val_loss: 0.0534 - val_acc: 0.9808
Epoch 2/8
119678/119678 [==============================] - 140s 1ms/step - loss: 0.0547 - acc: 0.9804 - val_loss: 0.0499 - val_acc: 0.9818
Epoch 3/8
119678/119678 [==============================] - 140s 1ms/step - loss: 0.0513 - acc: 0.9811 - val_loss: 0.0475 - val_acc: 0.9828
Epoch 4/8
119678/119678 [==============================] - 140s 1ms/step - loss: 0.0489 - acc: 0.9818 - val_loss: 0.0461 - val_acc: 0.9834
Epoch 5/8
119678/119678 [==============================] - 140s 1ms/step - loss: 0.0470 - acc: 0.9824 - val_loss: 0.0445 - val_acc: 0.9835
Epoch 6/8
119678/119678 [==============================] - 140s 1ms/step - loss: 0.0458 - acc: 0.9826 - val_loss: 0.0438 - val_acc: 0.9836
Epoch 7/8
119678/119678 [==============================] - 140s 1ms/step - loss: 0.0451 - acc: 0.9828 - val_loss: 0.0436 - val_acc: 0.9838
Epoch 8/8
119678/119678 [============